## Задание

Аугментировать данные техниками из занятия (albumentations, torchvision, imgaug...)


In [5]:
!pip install -q -U albumentations==1.2.0
!pip install git+https://github.com/mjkvaak/ImageDataAugmentor

     |████████████████████████████████| 113 kB 28.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/mjkvaak/ImageDataAugmentor to /tmp/pip-req-build-dbabrvk2
  Running command git clone -q https://github.com/mjkvaak/ImageDataAugmentor /tmp/pip-req-build-dbabrvk2
  Created wheel for ImageDataAugmentor: filename=ImageDataAugmentor-0.0.0-py3-none-any.whl size=29643 sha256=3d66e222587aa1373a96d164b70c00d360ef96259d59a03223503898ab9adfa1
  Stored in directory: /tmp/pip-ephem-wheel-cache-9lzv5tjt/wheels/c9/bd/73/9cfa59d2393dae55bbcc30f5aa901f55fe531c66efebbc8fc3
Successfully built ImageDataAugmentor


In [7]:
import numpy as np
import os
from pathlib import Path
import pandas as pd
import numpy as np
from statistics import mean
import tensorflow as tf
from keras.callbacks import History
from ImageDataAugmentor.image_data_augmentor import *
from albumentations import (
    Compose, RandomBrightness, JpegCompression, HueSaturationValue, RandomContrast, HorizontalFlip,
    Rotate
)

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.applications import mobilenet
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [9]:
dataset_path = "imagenette2"
if not os.path.exists(dataset_path):
    !wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
    !tar -xzf imagenette2.tgz

--2022-11-26 08:32:27--  https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.134.184, 54.231.171.48, 54.231.192.72, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.134.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1557161267 (1.5G) [application/x-tar]
Saving to: ‘imagenette2.tgz’

imagenette2.tgz     100%[===================>]   1.45G  12.5MB/s    in 1m 58s  

2022-11-26 08:34:27 (12.6 MB/s) - ‘imagenette2.tgz’ saved [1557161267/1557161267]



Подгружаем датасет

In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_path = Path('/content/imagenette2/train/')
val_path = Path('/content/imagenette2/val/')
accuracy = []

Ф-ция top 5 accuracy:

In [11]:
def top_k_accuracy(y_pred, y_true, k):
  true = 0
  for idx in range(len(y_true)):    
    if y_true[idx] in y_pred[idx][:k]:
      true += 1  
  return (true / len(y_true))

In [12]:
model = mobilenet.MobileNet(weights='imagenet')
helper_module = mobilenet

17225924/17225924 [==============================] - 2s 0us/step


In [13]:
valid_datagen = ImageDataGenerator(preprocessing_function=helper_module.preprocess_input)
valid_gen = valid_datagen.flow_from_directory(
  val_path,
  shuffle=False, seed=42, 
  target_size=model.input_shape[1:-1],
  batch_size=64
)  

Found 3925 images belonging to 10 classes.


Вычисляем top 5 accuracy

In [14]:
def model_top_5_accuracy(model, valid_gen):

  top_k_acc = [] 
  count = 1
  for batch_x, batch_y in valid_gen:
    preds = model.predict(batch_x, steps=1, verbose=0)
    preds_decoded = helper_module.decode_predictions(preds, top=5)
    preds_decoded = np.array(preds_decoded)
    preds_decoded.shape

    y_true_indexs = np.where(batch_y == 1)[1]
    y_true_labels = []
    for idx in y_true_indexs:
       y_true_labels.append(list(valid_gen.class_indices.items())[idx][0])

    top_k_acc.append(top_k_accuracy(preds_decoded, y_true_labels, 5))

    count += 1
    if count > valid_gen.__len__():
      break
    
  return mean(top_k_acc)


In [15]:
accuracy.append(model_top_5_accuracy(model, valid_gen))

35363/35363 [==============================] - 0s 0us/step


In [16]:
AUGMENTATIONS1 = albumentations.Compose([
    albumentations.Flip(p=0.5),
    albumentations.GaussianBlur(p=0.05),
])

AUGMENTATIONS2 = albumentations.Compose([
    albumentations.Flip(p=0.5),
    albumentations.GaussianBlur(p=0.05),
    albumentations.RandomCrop(height=192, width=192),
    albumentations.HueSaturationValue(p=0.5),
])

AUGMENTATIONS3 = albumentations.Compose([
    albumentations.RandomCrop(height=192, width=192),
    albumentations.OneOf([
        albumentations.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3),
        albumentations.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1)
    ],p=1),
    albumentations.HueSaturationValue(p=0.5),
])

In [17]:
def update_model_with_fc_layer(base_model):
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(1000, activation='relu')(x)
  predictions = Dense(10, activation='softmax')(x)
  model = Model(inputs=base_model.input, outputs=predictions)
  for layer in base_model.layers:
    layer.trainable = False
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tf.keras.metrics.TopKCategoricalAccuracy(k=5)])
  return model

In [18]:
base_model = mobilenet.MobileNet(weights='imagenet', include_top=False)
model = update_model_with_fc_layer(base_model)


17225924/17225924 [==============================] - 2s 0us/step


In [19]:
train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment=AUGMENTATIONS1,
        preprocess_input=None)
train_generator = train_datagen.flow_from_directory(
        train_path,
        shuffle=True, seed=42, 
        #target_size=model.input_shape[1:-1],
        batch_size=64)
val_datagen = ImageDataAugmentor(rescale=1./255)
validation_generator = val_datagen.flow_from_directory(
        val_path,
        shuffle=False, seed=42, 
        #target_size=model.input_shape[1:-1],
        batch_size=64)

/usr/local/lib/python3.7/dist-packages/ImageDataAugmentor/image_data_augmentor.py:333: UserWarning: Passing `seed` in `.flow_from_directory` has been been removed: pass  `seed` as parameter in `ImageDataAugmentor(..., seed=...)` instead
  warnings.warn('Passing `seed` in `.flow_from_directory` has been been removed: pass  `seed` '


Found 9469 images belonging to 10 classes.
Found 3925 images belonging to 10 classes.


In [20]:
history = model.fit(
  train_generator,
  steps_per_epoch=len(train_generator),
  epochs=10,
  validation_data=validation_generator,
  validation_steps=len(validation_generator))

Epoch 1/10
148/148 [==============================] - 102s 676ms/step - loss: 0.2253 - top_k_categorical_accuracy: 0.9879 - val_loss: 0.0526 - val_top_k_categorical_accuracy: 1.0000
Epoch 2/10
148/148 [==============================] - 99s 675ms/step - loss: 0.0798 - top_k_categorical_accuracy: 0.9996 - val_loss: 0.0374 - val_top_k_categorical_accuracy: 1.0000
Epoch 3/10
148/148 [==============================] - 100s 683ms/step - loss: 0.0721 - top_k_categorical_accuracy: 0.9995 - val_loss: 0.0484 - val_top_k_categorical_accuracy: 1.0000
Epoch 4/10
148/148 [==============================] - 100s 679ms/step - loss: 0.0570 - top_k_categorical_accuracy: 0.9995 - val_loss: 0.0638 - val_top_k_categorical_accuracy: 0.9995
Epoch 5/10
148/148 [==============================] - 100s 682ms/step - loss: 0.0619 - top_k_categorical_accuracy: 0.9996 - val_loss: 0.0502 - val_top_k_categorical_accuracy: 1.0000
Epoch 6/10
148/148 [==============================] - 99s 676ms/step - loss: 0.0500 - top_k

In [22]:
accuracy.append(mean(history.history['val_top_k_categorical_accuracy']))

In [23]:
train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment=AUGMENTATIONS2,
        preprocess_input=None)
train_generator = train_datagen.flow_from_directory(
        train_path,
        shuffle=True, seed=42, 
        #target_size=model.input_shape[1:-1],
        batch_size=64)
val_datagen = ImageDataAugmentor(rescale=1./255)
validation_generator = val_datagen.flow_from_directory(
        val_path,
        shuffle=False, seed=42, 
        #target_size=model.input_shape[1:-1],
        batch_size=64)

Found 9469 images belonging to 10 classes.
Found 3925 images belonging to 10 classes.


In [24]:
base_model = mobilenet.MobileNet(weights='imagenet', include_top=False)
update_model_with_fc_layer(base_model)

In [25]:
history = model.fit(
  train_generator,
  steps_per_epoch=len(train_generator),
  epochs=10,
  validation_data=validation_generator,
  validation_steps=len(validation_generator))

Epoch 1/10
148/148 [==============================] - 95s 642ms/step - loss: 0.1208 - top_k_categorical_accuracy: 0.9993 - val_loss: 0.0551 - val_top_k_categorical_accuracy: 1.0000
Epoch 2/10
148/148 [==============================] - 94s 636ms/step - loss: 0.0923 - top_k_categorical_accuracy: 0.9995 - val_loss: 0.0533 - val_top_k_categorical_accuracy: 0.9995
Epoch 3/10
148/148 [==============================] - 95s 644ms/step - loss: 0.0814 - top_k_categorical_accuracy: 0.9994 - val_loss: 0.0595 - val_top_k_categorical_accuracy: 1.0000
Epoch 4/10
148/148 [==============================] - 93s 634ms/step - loss: 0.0715 - top_k_categorical_accuracy: 0.9998 - val_loss: 0.0449 - val_top_k_categorical_accuracy: 0.9995
Epoch 5/10
148/148 [==============================] - 93s 636ms/step - loss: 0.0615 - top_k_categorical_accuracy: 0.9996 - val_loss: 0.0339 - val_top_k_categorical_accuracy: 1.0000
Epoch 6/10
148/148 [==============================] - 94s 636ms/step - loss: 0.0626 - top_k_cat

In [26]:
accuracy.append(mean(history.history['val_top_k_categorical_accuracy']))

In [27]:
train_datagen = ImageDataAugmentor(
        rescale=1./255,
        augment=AUGMENTATIONS3,
        preprocess_input=None)
train_generator = train_datagen.flow_from_directory(
        train_path,
        shuffle=True, seed=42, 
        #target_size=model.input_shape[1:-1],
        batch_size=64)
val_datagen = ImageDataAugmentor(rescale=1./255)
validation_generator = val_datagen.flow_from_directory(
        val_path,
        shuffle=False, seed=42, 
        #target_size=model.input_shape[1:-1],
        batch_size=64)

Found 9469 images belonging to 10 classes.
Found 3925 images belonging to 10 classes.


In [28]:
base_model = mobilenet.MobileNet(weights='imagenet', include_top=False)
update_model_with_fc_layer(base_model)

In [29]:
history = model.fit(
  train_generator,
  steps_per_epoch=len(train_generator),
  epochs=10,
  validation_data=validation_generator,
  validation_steps=len(validation_generator))

Epoch 1/10
148/148 [==============================] - 98s 664ms/step - loss: 0.0247 - top_k_categorical_accuracy: 0.9998 - val_loss: 0.0459 - val_top_k_categorical_accuracy: 0.9997
Epoch 2/10
148/148 [==============================] - 110s 746ms/step - loss: 0.0220 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0440 - val_top_k_categorical_accuracy: 0.9997
Epoch 3/10
148/148 [==============================] - 97s 658ms/step - loss: 0.0194 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0540 - val_top_k_categorical_accuracy: 0.9997
Epoch 4/10
148/148 [==============================] - 98s 665ms/step - loss: 0.0185 - top_k_categorical_accuracy: 0.9999 - val_loss: 0.0460 - val_top_k_categorical_accuracy: 0.9995
Epoch 5/10
148/148 [==============================] - 98s 664ms/step - loss: 0.0161 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.0484 - val_top_k_categorical_accuracy: 0.9997
Epoch 6/10
148/148 [==============================] - 109s 742ms/step - loss: 0.0154 - top_k_c

In [30]:
accuracy.append(mean(history.history['val_top_k_categorical_accuracy']))

In [31]:
augmentations = ["Without augmentations", "Flip, GaussianBlur", "Flip, GaussianBlur, RandomCrop, HueSaturation", "RandomCrop, BrightnessContrast, HueSaturation"]

In [32]:
d = {'Augmentations': augmentations, 'Top 5 accuracy': accuracy}
df = pd.DataFrame(data=d)
df

,Augmentations,Top 5 accuracy
0,Without augmentations,0.980583
1,"Flip, GaussianBlur",0.999924
2,"Flip, GaussianBlur, RandomCrop, HueSaturation",0.999796
3,"RandomCrop, BrightnessContrast, HueSaturation",0.999771
